In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split

Here I'll split data into ~40K train images and ~5K holdout set.

In [2]:
with open('../data/styles-44c73c23.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.head()

,id,price,discountedPrice,styleType,productTypeId,articleNumber,visualTag,productDisplayName,variantName,myntraRating,...,lookGoodAlbum,style360Images,masterCategory,subCategory,articleType,isEMIEnabled,productDescriptors,styleOptions,colours,discountData
0,1566,299,299,P,299,27144,,Artengo Men Black Cap,Black,10,...,{},{},Accessories,Headwear,Caps,False,-,"[{'id': 4953, 'name': 'Size', 'value': 'JUNIOR...",NaN,NaN
1,39342,2199,2199,P,288,81SU292201,,Fila Men Zoom Black Shoes,ZOOM HI,0,...,{},{},Footwear,Shoes,Casual Shoes,True,<p><strong>Style Note</strong><br />A perfect ...,"[{'id': 157461, 'name': 'Size', 'value': 'UK4'...",{'colors': {'39341': {'dre_landing_page_url': ...,NaN
2,13182,2499,2499,P,288,1SB106XX010,,FILA Men Dls Maxim Black Sports Shoes,Dls Maxim,1,...,{},{},Footwear,Shoes,Sports Shoes,True,"<p style=""text-align: justify;"">Win those winn...","[{'id': 61037, 'name': 'Size', 'value': 'UK7',...",{'colors': {'13181': {'dre_landing_page_url': ...,NaN
3,21762,1695,1440,RTV,360,MB1212001,,Fossil Men Mead Black Belt,Mead,0,...,{},{},Accessories,Belts,Belts,True,"<p>Crafted from pure leather, this fossil belt...","[{'id': 98334, 'name': 'Size', 'value': '36', ...",NaN,"{'discountType': 1, 'discountAmount': 255, 'di..."
4,48692,1525,1509,P,416,EF8782,,Lucera Women Silver Earrings,Silver,1,...,{},{},Accessories,Jewellery,Earrings,False,<p>Silver coloured earrings<br />An amber colo...,"[{'id': 189222, 'name': 'Size', 'value': 'Ones...",NaN,"{'discountType': 1, 'discountAmount': 16, 'dis..."


In [4]:
embeddings = pd.read_parquet('../data/embeddings-efe5f026.parquet')

In [5]:
df = df[df['id'].isin(embeddings['image'])]

In [6]:
df.subCategory.value_counts()

Topwear                     15401
Shoes                        7344
Bags                         3055
Bottomwear                   2693
Watches                      2542
Innerwear                    1808
Jewellery                    1080
Eyewear                      1073
Fragrance                    1012
Sandal                        963
Wallets                       933
Flip Flops                    915
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Accessories                   143
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Skin Care                      77
Skin          

In [7]:
# rename some categories

df.subCategory = df.subCategory.replace('Sports Accessories', 'Sports Equipment')
df.subCategory = df.subCategory.replace('Skin Care', 'Skin')

In [8]:
df.subCategory.value_counts()

Topwear                     15401
Shoes                        7344
Bags                         3055
Bottomwear                   2693
Watches                      2542
Innerwear                    1808
Jewellery                    1080
Eyewear                      1073
Fragrance                    1012
Sandal                        963
Wallets                       933
Flip Flops                    915
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Skin                          146
Accessories                   143
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Eyes          

In [9]:
v_counts = df.subCategory.value_counts()

In [10]:
df = df[df.subCategory.isin(v_counts[v_counts > 10].index)]
df.subCategory.value_counts()

Topwear                     15401
Shoes                        7344
Bags                         3055
Bottomwear                   2693
Watches                      2542
Innerwear                    1808
Jewellery                    1080
Eyewear                      1073
Fragrance                    1012
Sandal                        963
Wallets                       933
Flip Flops                    915
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Skin                          146
Accessories                   143
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Eyes          

In [13]:
cat_cols = df.dtypes[df.dtypes == 'category'].index
df[cat_cols] = df[cat_cols].astype('object').astype('category')

In [14]:
df.subCategory.value_counts()

Topwear                     15401
Shoes                        7344
Bags                         3055
Bottomwear                   2693
Watches                      2542
Innerwear                    1808
Jewellery                    1080
Eyewear                      1073
Fragrance                    1012
Sandal                        963
Wallets                       933
Flip Flops                    915
Belts                         811
Socks                         698
Lips                          527
Dress                         478
Loungewear and Nightwear      470
Saree                         427
Nails                         329
Makeup                        307
Headwear                      293
Ties                          258
Skin                          146
Accessories                   143
Scarves                       118
Cufflinks                     108
Apparel Set                   106
Free Gifts                    104
Stoles                         90
Eyes          

In [15]:
df.shape

(44412, 40)

In [16]:
embeddings = embeddings[embeddings['image'].isin(df['id'])]
embeddings.shape

(44412, 1281)

In [17]:
X = df.drop(['subCategory'], axis=1)
y = df['subCategory']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [18]:
holdout_ids = X_test.id
holdout_ids

1173     23591
41654    39604
12891    49462
38182    26809
25148    56261
         ...  
21048    47450
6375     52897
37723     4509
42660    55090
19690    13843
Name: id, Length: 8883, dtype: object

In [19]:
holdout_ids.to_csv('../data/holdout_ids.csv', index=False)

In [21]:
with open('../data/styles-44c73c23.pkl', 'wb') as f:
    pickle.dump(df, f)

In [22]:
embeddings.to_parquet('../data/embeddings-44c73c23.parquet')